### Results 

This notebook reproduced the analyses reported in the paper "Risky looks: Visual cues that predict intuitive HIV risk perception". 

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(font_scale=1.2)
from sklearn import datasets
from sklearn.model_selection import cross_val_predict, cross_val_score #KFold
from sklearn.cross_validation import KFold  #using old interface here
from sklearn import linear_model

os.chdir('../data')

### Relationships between individual cues and perceptions of HIV risk

First, data are read in and a correlation between all cues, and particularly between each cue and the criterion variable "HIV risk" is computed, storing the results in a dataframe. Moreover, we compute basic descriptives for each cue, i.e. the mean (on a 1-7 scale) and standard deviation of ratings across the 240 images.

In [ ]:
full_dataset = pd.read_csv('FullSet_240.txt', delimiter='\t')
risk_correlation_dataset = full_dataset.drop([   'BildNr','No','Sex',
               'Trust','Health','Attractiveness',
                           ' CUE3_Appearance3_Unshaved (vs. shaved)_Abstract_0',
                           ' CUE34_Face17_Strong (vs. no) growth of beard_Abstract_0',
                           ' CUE53_Mouth2_Yellow (vs. white) teeth_Abstract_0',
                           ' CUE62_Set8_Dark (vs. well-lit) background_Abstract_0',
                           ' CUE35_GenImp1_Self confident (vs. not)_Abstract_2',
                           ' CUE36_GenImp2_Cautious (vs. risk-seeking)_Abstract_2',
                           ' CUE37_GenImp3_Uneducated (vs. educated)_Abstract_2',
                           ' CUE38_GenImp4_Southern (vs. nordic) type_Abstract_1',
                           ' CUE39_GenImp5_Sporty (vs. unsporty)_Abstract_2',
                           ' CUE40_GenImp6_Scruffy (vs. kempt)_Abstract_2',
                           ' CUE41_GenImp7_Likeable (vs. unsympathetic)_Abstract_2',
                           ' CUE42_GenImp8_Attractive (vs. unattractive)_Abstract_2',
                           ' CUE43_GenImp9_Irresponsible (vs. responsible)_Abstract_2',
                           ' CUE44_GenImp10_Homosexual (vs. heterosexual)_Abstract_2',
                           ' CUE45_GenImp11_Popular (vs. unpopular)_Abstract_2',
                           ' CUE46_GenImp12_Selfish (vs. unselfish)_Abstract_2',
                           ' CUE47_GenImp13_Ill-looking (vs. healthy-looking)_Abstract_2'], axis=1)

# Computing correlations
risk_cue_correlations = risk_correlation_dataset.corr()

# Setting up a dataframe to store the risk-cue-correlations (just a column in the results matrix)
results_table = pd.DataFrame(np.round(risk_cue_correlations.Risk[1:].values, 2) , columns = ['risk_cue_correlations'])

# giving the results table the proper cue names/labels
results_table.index = risk_cue_correlations.index[1:,]

# computing cue means and sds, and putting them into the results_table
results_table['Cue Means'] = np.round(risk_correlation_dataset.describe().loc['mean'][1:].values, 2)
results_table['Cue Stds'] = np.round(risk_correlation_dataset.describe().loc['std'][1:].values,2)
results_table

In [ ]:
# Uncomment to visually inspect relationships between cues
#df = risk_correlation_dataset.iloc[:, 2:10]
#df.columns = np.arange(2,10,1)
#sns.pairplot(df, kind='reg');

### Predicting HIV risk impressions from visual cues
#### Set up dataset for regression modeling 

After assessing correlations between each cue and HIV risk, the next step is setting up a predictive model to forecast HIV risk from the combination of cues. We begin by creating a dataset (X) and the outcome variable (y - i.e. HIV risk).

In [ ]:
# defining datamatrix and outcome vector
X = risk_correlation_dataset
y = X['Risk']
X = X.drop([ 'Risk'], axis=1)
X

#### Set up Lasso, cross-validate, plot

This is the core analysis that sets up a lasso-regression model and estimates it against the data in a 10-fold cross-validation. Results are then plotted and metrics are assessed.

In [ ]:
# Defining the LassoEstimator
lasso = linear_model.LassoCV()

# predict the data
predicted = cross_val_predict(lasso, X, y, cv=10)

# plot
fig, ax = plt.subplots(figsize = (5, 5))
ax.scatter(y, predicted, color = 'black')
ax.set_xlabel('Actual impressions of HIV risk')
ax.set_ylabel('Predicted impression of HIV risk')
ax.set_xlim((1,7))
ax.set_ylim((1,7))
#ax.set_title('Result of Lasso Regression')
plt.show()

# give correlation between measured and predicted results
corr_m2p = np.corrcoef(y, predicted)[0,1]
corr_m2p = np.round(corr_m2p, 2)
print( 'Correlation measured2predicted: '  + str( corr_m2p ) )

# compute x-val score r2 and mean thereof
scores = cross_val_score(lasso, X, y, cv=10, scoring = 'r2')
m_r2 = np.mean(scores)
m_r2 = np.round(m_r2, 2)
print('Mean R2: ' + str( m_r2 ) )

# computing the standard error of the estimate 
se = np.sqrt(np.sum((y-predicted)**2)/len(y))
se = np.round(se, 2)
print('Standard error of the estimate: ' + str( se ) )


# get lasso coefficients
lasso_model = lasso.fit(X, y)
len(lasso_model.coef_)

results_table['Lasso Coefficients'] = np.round(lasso_model.coef_, 2)
#results_table
results_table.to_csv('results_table.csv')

#### Inspecting the Lasso coefficients

In addition to know that the model predicts HIV risk quite well, we want to know which cues are particularly important. We do so by sorting the cue-coefficients

In [ ]:
sorted = results_table.sort_values('Lasso Coefficients', ascending=False)
print('Top 5')
print(sorted['Lasso Coefficients'][:5])
print('-----')
print('Bottom 5')
sorted['Lasso Coefficients'][-5:]

In [ ]:
sorted = results_table.sort_values('Lasso Coefficients', ascending=False)
print('Top 5')
print(sorted['Lasso Coefficients'][:5])
print('-----')
print('Bottom 5')
sorted['Lasso Coefficients'][-5:]

#### Setting up a conventional regression model
The above results rely on Lasso-regression. The classical approach would be a standard linear regression. Due to multicolinearity, this model is not advisable, but in this case does not markedly change the results.

In [ ]:
lr = linear_model.LinearRegression()
predicted = cross_val_predict(lr, X, y, cv=10)

fig, ax = plt.subplots(figsize = (5, 5))
ax.scatter(y, predicted, color = 'black')
ax.set_xlabel('Actual impressions of HIV risk')
ax.set_ylabel('Predicted impression of HIV risk')
ax.set_xlim((1,7))
ax.set_ylim((1,7))
ax.set_title('Result of Linear Regression')
plt.show()

# give correlation between measured and predicted results
corr_m2p = np.corrcoef(y, predicted)[0,1]
corr_m2p = np.round(corr_m2p, 2)
print( 'Correlation measured2predicted: '  + str( corr_m2p ) )

# compute x-val score r2 and mean thereof
scores = cross_val_score(lr, X, y, cv=10, scoring = 'r2')
m_r2 = np.mean(scores)
m_r2 = np.round(m_r2, 2)
print('Mean R2: ' + str( m_r2 ) )

# computing the standard error of the estimate 
se = np.sqrt(np.sum((y-predicted)**2)/len(y))
se = np.round(se, 2)
print('Standard error of the estimate: ' + str( se ) )

# get lasso coefficients
lr_model = lr.fit(X, y)
len(lr_model.coef_)

results_table['LR Coefficients'] = np.round(lr_model.coef_, 2)
#results_table

#### Reducing the cues to concrete  ones and running same LASSO model

Inspecting the models above reveals that some of the most informative cues are themselves abstractions (e.g. "unconventional appearance"). This is expected since HIV risk is likely a "compound impression" and not neatly tied to e.g. the brightness of the image. However, to examine how the results change if we confine the data to concrete (clearly observable) cues, we run the same procedures using a reduced dataset.

In [ ]:
n_cues = len(X.columns.values)
abstraction_selector = np.ones(n_cues)
for run in range(n_cues):
    if (int(X.columns.values[run][-1]) > 0) :
            abstraction_selector[run] = 0

X_concrete = X.iloc[:, np.where(abstraction_selector > 0)[0]  ]

In [ ]:
# predict the data
predicted = cross_val_predict(lasso, X_concrete, y, cv=10)

# plot
fig, ax = plt.subplots(figsize = (5, 5))
ax.scatter(y, predicted, color = 'black')
ax.set_xlabel('Actual impressions of HIV risk')
ax.set_ylabel('Predicted impression of HIV risk')
ax.set_xlim((1,7))
ax.set_ylim((1,7))
ax.set_title('Result of Lasso Regression - Concrete Cue Set')
plt.show()

# give correlation between measured and predicted results
corr_m2p = np.corrcoef(y, predicted)[0,1]
corr_m2p = np.round(corr_m2p, 2)
print( 'Correlation measured2predicted: '  + str( corr_m2p ) )

# compute x-val score r2 and mean thereof
scores = cross_val_score(lasso, X_concrete, y, cv=10, scoring = 'r2')
m_r2 = np.mean(scores)
m_r2 = np.round(m_r2, 2)
print('Mean R2: ' + str( m_r2 ) )

# computing the standard error of the estimate 
se = np.sqrt(np.sum((y-predicted)**2)/len(y))
se = np.round(se, 2)
print('Standard error of the estimate: ' + str( se ) )


# get lasso coefficients
lasso_model_reduced = lasso.fit(X_concrete, y)
len(lasso_model_reduced.coef_)

results_table_reduced = pd.DataFrame(  index= X_concrete.columns.values)

results_table_reduced['Lasso Coefficients ConcreteOnly'] = np.round(lasso_model_reduced.coef_, 2)
#results_table

In [ ]:
sorted = results_table_reduced.sort_values('Lasso Coefficients ConcreteOnly', ascending=False)
print('Top 5')
print(sorted['Lasso Coefficients ConcreteOnly'][:5])
print('-----')
print('Bottom 5')
sorted['Lasso Coefficients ConcreteOnly'][-5:]

### How does HIV risk perception relate to general person impressions?

In [ ]:
full_dataset
impression_dataset = full_dataset[['Risk', 
                                   ' CUE35_GenImp1_Self confident (vs. not)_Abstract_2',
                                   ' CUE36_GenImp2_Cautious (vs. risk-seeking)_Abstract_2',
                                   ' CUE37_GenImp3_Uneducated (vs. educated)_Abstract_2',
                                   ' CUE38_GenImp4_Southern (vs. nordic) type_Abstract_1',
                                   ' CUE39_GenImp5_Sporty (vs. unsporty)_Abstract_2',
                                   ' CUE40_GenImp6_Scruffy (vs. kempt)_Abstract_2',
                                   ' CUE41_GenImp7_Likeable (vs. unsympathetic)_Abstract_2',
                                   ' CUE42_GenImp8_Attractive (vs. unattractive)_Abstract_2',
                                   ' CUE43_GenImp9_Irresponsible (vs. responsible)_Abstract_2',
                                   ' CUE44_GenImp10_Homosexual (vs. heterosexual)_Abstract_2',
                                   ' CUE45_GenImp11_Popular (vs. unpopular)_Abstract_2',
                                   ' CUE46_GenImp12_Selfish (vs. unselfish)_Abstract_2',
                                   ' CUE47_GenImp13_Ill-looking (vs. healthy-looking)_Abstract_2',]
                                 ]

risk_impressions_correlations = impression_dataset.corr()

# Setting up a dataframe to store the risk-cue-correlations (just a column in the results matrix)
impression_results_table = pd.DataFrame(np.round(risk_impressions_correlations.Risk[1:].values, 2) , columns = ['risk_impressions_correlations'])



# giving the results table the proper cue names/labels
impression_results_table.index = risk_impressions_correlations.index[1:,]

# computing cue means and sds, and putting them into the results_table
impression_results_table['Mean Rating'] = np.round(risk_impressions_correlations.describe().loc['mean'][1:].values, 2)
impression_results_table['Std'] = np.round(risk_impressions_correlations.describe().loc['std'][1:].values,2)
impression_results_table
impression_results_table.to_csv('impression_results_table.csv')

### The relationship between impressions of HIV risk to perceptions of trust, health,  and attractiveness.

#### Correlations between variables
First, we can just examine the correlations between HIV risk and other dimensions of social impressions.

In [ ]:
variables = [ 'Risk',
              'Trust',
              'Health',
              'Attractiveness']

full_dataset[variables].corr()   

#### Create analogous models for Trust, Health, Attractiveness as for HIV risk

Next, just as we created models for HIV risk above, we can train analogous models for different criterion variables.

In [ ]:
# use the same cue-dataset to train a model for the other variables
X = risk_correlation_dataset.drop([ 'Risk'], axis=1) 

for curr_var in variables:
    #print(curr_var)
    y = full_dataset[curr_var]
    
    scores = cross_val_score(lasso, X, y, cv=5, scoring = 'r2')
    m_r2 = np.mean(scores)
    m_r2 = np.round(m_r2, 2)
    print(curr_var +  '-Mean R2: ' + str( m_r2 ) )

    predicted = cross_val_predict(lasso, X, y, cv=10)

    # give correlation between measured and predicted results
    corr_m2p = np.corrcoef(y, predicted)[0,1]
    corr_m2p = np.round(corr_m2p, 2)
    print( curr_var + ' - Correlation measured2predicted: '  + str( corr_m2p ) )

#### Use models fitted on other Trust/Health/Attractiveness to predict HIV risk in new data

Finally, we can train models to predict trust (or health, or attractiveness), and test how well these models would predict HIV risk. Of note, the behavior of these models can be guessed based on the relationships between these variables in the correlation table above. However, it could in principle be that there is a relationship between the variables (e.g. Risk and Trust), but that this reltionship arises from an entirely different cue-utilization process. In classical lens-model style analyses, this would be revealed by so-called vector-correlations, i.e. correlations between the cue-Risk correlation vector and the cue-Trust correlation vector. Our predictive modeling approach is really just another way to look at this phenomenon, i.e. whether same or different cues may drive the judgments. I would argue, though, that it has the advantage of being cross-validated.

In [ ]:
X = risk_correlation_dataset.drop(['Risk'], axis=1) 
X = full_dataset[variables].join(X)

for curr_var in variables[1:]:
    print(curr_var)
    
    res_var_from_var = []
    res_risk_from_var = []

    
    # Iterating the KFold object returns tuples of
    # randomly shuffled training and testing indices, 
    # which are then used to select relevant data and labels/outcomes
    for train, test in KFold(len(X), 10, shuffle=True):   #K = 5
    
        X_train = X.iloc[train]
        X_test = X.iloc[test]

        y_train = X_train[curr_var]
        y_test_var = X_test[curr_var]
        y_test_risk = X_test['Risk']

        X_train = X_train.drop(variables, axis=1)
        X_test = X_test.drop(variables, axis=1)

        lasso_model = lasso.fit(X_train, y_train)
        y_var_predicted = lasso_model.predict(X_test)

        res_var_from_var.append(np.corrcoef(y_var_predicted, y_test_var)[0,1])
        res_risk_from_var.append(np.corrcoef(y_var_predicted, y_test_risk)[0,1])

    print(np.mean(res_var_from_var))    
    print(np.mean(res_risk_from_var))
    plt.plot(res_var_from_var)
    plt.plot(res_risk_from_var)
    #plt.plot([x * -1 for x in res_risk_from_var])
    plt.title(curr_var)
    leg1 = curr_var + " from " + curr_var + '-Model'
    leg2 = 'HIV risk from ' + curr_var + '-Model'
    plt.legend( [leg1, 
                 leg2],
              loc = 'lower right')
    plt.ylim(-1,1)
    plt.show()
                   

In [ ]:
print("Done!")